In [ ]:
import numpy as np
from scipy.linalg import expm
import pandas as pd
from scipy.sparse import diags

import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font to Times New Roman for the whole plot
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

## Functions

In [ ]:
# Optimal controls verification
def optimal_controls(t,T,
                     N_1,zeta,
                     lambda_a,lambda_b,alpha1,phi1,kappa1,
                     Y_1,hat_y1,m2_1,y,
                     delta_a_2,delta_b_2):
    
    K=k_matrix(N_1, lambda_a, lambda_b, phi1, kappa1, Y_1, hat_y1, delta_a_2, delta_b_2)
    w=W_t_function(alpha1, kappa1, zeta, Y_1, hat_y1, K, t)
    
    
    h1_1st=h1_function(zeta,kappa1,w,y,Y_1);
    h1_2nd=h1_function(zeta,kappa1,w,y+zeta,Y_1);
    h1_3rd=h1_function(zeta,kappa1,w,y-zeta,Y_1);
    z= m2_1/(y**2);
    delta_b_star = (1 / (2 * kappa1)) + (h1_1st - h1_2nd) / zeta
    delta_a_star = (1 / (2 * kappa1)) + (h1_1st - h1_3rd) / zeta
    return delta_b_star, delta_a_star

In [ ]:
# Level functions and derivative
def lvlfct(m_2, y):
    return m_2/y;

def lvlfct_deriv(m_2,y):
    return -m_2/y**2

In [ ]:
# h1 and H1
def h1_function(zeta, kappa, w, y, Y_1):
    # Find entries in w where Y_1 matches y
    w=np.sqrt(w)
    matched_w = w[Y_1 == y]
    # Check if the matched_w array is empty
    if matched_w.size == 0:
        return np.nan  # Return np.nan if y is not found in Y_1
    else:
        # Calculate h1 if there is at least one match
        h1 = (zeta / kappa) * np.log(matched_w.item())  # Assuming only one match; use matched_w[0] if multiple matches are handled differently
        return h1
    
def H1_function(x,y,z,h1):
    H1=x+y*z+h1
    return H1

In [ ]:
# W_t
def W_t_function(alpha1, kappa1, zeta, Y_1, hat_y1, K, t):
    exponent_values = -alpha1 * kappa1 / zeta * (Y_1 - hat_y1)**2
    W0 = (np.exp(exponent_values)) **2
    W_t = expm(K * (T-t)).dot(W0)
    return W_t

In [ ]:
# K matrix
def k_matrix(N_1, lambda_a, lambda_b, phi1, kappa1, Y_1, hat_y1, delta_a_2, delta_b_2):
    # The main diagonal values
    main_diag = -2 * kappa1 * phi1 * (Y_1 - hat_y1)**2 / zeta

    # Subdiagonal and superdiagonal values
    sub_diag = lambda_a * np.exp(kappa1 * delta_a_2 - 1) * np.ones(N_1)  
    super_diag = lambda_b * np.exp(kappa1 * delta_b_2 - 1) * np.ones(N_1)

    # Create the tridiagonal sparse matrix
    diagonals = [sub_diag, main_diag, super_diag]
    K = diags(diagonals, offsets=[-1, 0, 1], shape=(N_1+1, N_1+1), format='csr')
    
    return K.todense()

## Parameters and initial assumptions

In [ ]:
# AMM default parameters (Table 6.2)
zeta = 1 # Trading size [units of y]
T = 30 # Trading window [seconds]
lambda_a = 1 # Baseline sell order arrival intensity [order/second]
lambda_b = 1 # Baseline buy order arrival intensity [order/second]

In [ ]:
# Liquidity providers default characteristics (Table 6.1)
x0_1=1 # Initial x reserve for LP1
y0_1=1 # Initial y reserve for LP1
z0_1=x0_1/y0_1 # Initial marginal rate z for LP1
m2_1=x0_1*y0_1 # Liquidity depth measure for LP1

y_min_1 = -3 # Y lower reserve constraint for LP1
y_max_1 = 3 # Y upper reserve constraint for LP1
N_1=int((y_max_1-y_min_1)/zeta)
Y_1 = np.linspace(y_max_1, y_min_1, N_1+1)
Z_1 = m2_1 / (Y_1 ** 2 + 1e-10)  # Adding a small constant to avoid division by zero

phi1 = 10**(-5) # Inventory penalty for LP1
alpha1 = 0.001 # Terminal penalty for LP1
kappa1= 10 # Rate of order pressure decay for LP1
hat_y1 = 0 # Inventory objective for LP1

# LP2 reserves and settings
z_2=z0_1 # Initial marginal rate z for LP2
delta_a_2=0.5 # Arbritrary constant sell depth for LP2
delta_b_2=0.5 # Arbritrary constant buy depth for LP2

## Simulations

In [ ]:
time_steps = 100  # Number of time steps to evaluate
t_grid = np.linspace(0, T, time_steps)
y_test_range=[-3,-2,-1,1,2,3]
# Storing results for each time point (x-coordinate) and y (y-coordinate)
delta_b_star_1 = np.zeros((len(y_test_range),len(t_grid)))
delta_a_star_1 = np.zeros((len(y_test_range),len(t_grid)))
# Compute W(t) for each time point
for i,y_test in enumerate(y_test_range):
    for j, t in enumerate(t_grid):
        x_test=y_test
        z_test=x_test/y_test
        m2_1=x_test*y_test;
        z_2_test=z_test
        S=z_test
        
        delta_b_star_1[i,j], delta_a_star_1[i,j] = optimal_controls(t,T,
                     N_1,zeta,
                     lambda_a,lambda_b,alpha1,phi1,kappa1,
                     Y_1,hat_y1,m2_1,y_test,
                     delta_a_2,delta_b_2)

In [ ]:
phi11 = np.linspace(10**(-2),10**(-5)) # No inventory penalty
t=t_grid[76]
# Storing results for each time point (x-coordinate) and y (y-coordinate)
delta_b_star_2 = np.zeros((len(y_test_range),len(phi11)))
delta_a_star_2 = np.zeros((len(y_test_range),len(phi11)))
# Compute W(t) for each time point
for i,y_test in enumerate(y_test_range):
    for j, phi in enumerate(phi11):
        x_test=y_test
        z_test=x_test/y_test
        m2_1=x_test*y_test;
        z_2_test=z_test
        S=z_test
        
        delta_b_star_2[i,j], delta_a_star_2[i,j] = optimal_controls(t,T,
                     N_1,zeta,
                     lambda_a,lambda_b,alpha1,phi,kappa1,
                     Y_1,hat_y1,m2_1,y_test,
                     delta_a_2,delta_b_2)

In [ ]:
alpha11 = np.linspace(10**(-1),10**(-3)) # No terminal penalty
t=t_grid[-1]
# Storing results for each time point (x-coordinate) and y (y-coordinate)
delta_b_star_3 = np.zeros((len(y_test_range),len(alpha11)))
delta_a_star_3 = np.zeros((len(y_test_range),len(alpha11)))
# Compute W(t) for each time point
for i,y_test in enumerate(y_test_range):
    for j, alpha in enumerate(alpha11):
        x_test=y_test
        z_test=x_test/y_test
        m2_1=x_test*y_test;
        z_2_test=z_test
        S=z_test
        
        delta_b_star_3[i,j], delta_a_star_3[i,j] = optimal_controls(t,T,
                     N_1,zeta,
                     lambda_a,lambda_b,alpha,phi1,kappa1,
                     Y_1,hat_y1,m2_1,y_test,
                     delta_a_2,delta_b_2)

In [ ]:
kappa11= np.linspace(1,30)
t=t_grid[5]
# Storing results for each time point (x-coordinate) and y (y-coordinate)
delta_b_star_4 = np.zeros((len(y_test_range),len(kappa11)))
delta_a_star_4 = np.zeros((len(y_test_range),len(kappa11)))
# Compute W(t) for each time point
for i,y_test in enumerate(y_test_range):
    for j, kappa in enumerate(kappa11):
        x_test=y_test
        z_test=x_test/y_test
        m2_1=x_test*y_test;
        z_2_test=z_test
        S=z_test
        
        delta_b_star_4[i,j], delta_a_star_4[i,j] = optimal_controls(t,T,
                     N_1,zeta,
                     lambda_a,lambda_b,alpha1,phi1,kappa,
                     Y_1,hat_y1,m2_1,y_test,
                     delta_a_2,delta_b_2)

In [ ]:
plt.figure(figsize=(14,6))


plt.subplot(2,4,1)
plt.plot(t_grid,delta_a_star_1[5,:],'b',label='y=3')
plt.plot(t_grid,delta_a_star_1[4,:],'g',label='y=2')
plt.plot(t_grid,delta_a_star_1[3,:],'r',label='y=1')
plt.plot(t_grid,delta_a_star_1[2,:],'c',label='y=-1')
plt.plot(t_grid,delta_a_star_1[1,:],'violet',label='y=-2')
plt.plot(t_grid,delta_a_star_1[0,:],'y',label='y=-3')
plt.ylabel('Sell depth $\delta^{1,a}$')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02])  # Set specific y-axis ticks


plt.subplot(2,4,5)
plt.plot(t_grid,delta_b_star_1[5,:],'b',label='y=3')
plt.plot(t_grid,delta_b_star_1[4,:],'g',label='y=2')
plt.plot(t_grid,delta_b_star_1[3,:],'r',label='y=1')
plt.plot(t_grid,delta_b_star_1[2,:],'c',label='y=-1')
plt.plot(t_grid,delta_b_star_1[1,:],'violet',label='y=-2')
plt.plot(t_grid,delta_b_star_1[0,:],'y',label='y=-3')
plt.xlabel('Time (s)')
plt.ylabel('Buy depth $\delta^{1,b}$');
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02]);  # Set specific y-axis ticks

plt.subplot(2,4,2)
plt.plot(phi11,delta_a_star_2[5,:],'b',label='y=3')
plt.plot(phi11,delta_a_star_2[4,:],'g',label='y=2')
plt.plot(phi11,delta_a_star_2[3,:],'r',label='y=1')
plt.plot(phi11,delta_a_star_2[2,:],'c',label='y=-1')
plt.plot(phi11,delta_a_star_2[1,:],'violet',label='y=-2')
plt.plot(phi11,delta_a_star_2[0,:],'y',label='y=-3')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02])  # Set specific y-axis ticks


plt.subplot(2,4,6)
plt.plot(phi11,delta_b_star_2[5,:],'b',label='y=3')
plt.plot(phi11,delta_b_star_2[4,:],'g',label='y=2')
plt.plot(phi11,delta_b_star_2[3,:],'r',label='y=1')
plt.plot(phi11,delta_b_star_2[2,:],'c',label='y=-1')
plt.plot(phi11,delta_b_star_2[1,:],'violet',label='y=-2')
plt.plot(phi11,delta_b_star_2[0,:],'y',label='y=-3')
plt.xlabel('$\\phi^1$')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02]);  # Set specific y-axis ticks

plt.subplot(2,4,3)
plt.plot(alpha11,delta_a_star_3[5,:],'b',label='y=3')
plt.plot(alpha11,delta_a_star_3[4,:],'g',label='y=2')
plt.plot(alpha11,delta_a_star_3[3,:],'r',label='y=1')
plt.plot(alpha11,delta_a_star_3[2,:],'c',label='y=-1')
plt.plot(alpha11,delta_a_star_3[1,:],'violet',label='y=-2')
plt.plot(alpha11,delta_a_star_3[0,:],'y',label='y=-3')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02])  # Set specific y-axis ticks


plt.subplot(2,4,7)
plt.plot(alpha11,delta_b_star_3[5,:],'b',label='y=3')
plt.plot(alpha11,delta_b_star_3[4,:],'g',label='y=2')
plt.plot(alpha11,delta_b_star_3[3,:],'r',label='y=1')
plt.plot(alpha11,delta_b_star_3[2,:],'c',label='y=-1')
plt.plot(alpha11,delta_b_star_3[1,:],'violet',label='y=-2')
plt.plot(alpha11,delta_b_star_3[0,:],'y',label='y=-3')
plt.xlabel('$\\alpha^1$')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02]);  # Set specific y-axis ticks

plt.subplot(2,4,4)
plt.plot(kappa11,delta_a_star_4[5,:],'b',label='y=3')
plt.plot(kappa11,delta_a_star_4[4,:],'g',label='y=2')
plt.plot(kappa11,delta_a_star_4[3,:],'r',label='y=1')
plt.plot(kappa11,delta_a_star_4[2,:],'c',label='y=-1')
plt.plot(kappa11,delta_a_star_4[1,:],'violet',label='y=-2')
plt.plot(kappa11,delta_a_star_4[0,:],'y',label='y=-3')
plt.legend(loc='center right')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02])  # Set specific y-axis ticks


plt.subplot(2,4,8)
plt.plot(kappa11,delta_b_star_4[5,:],'b',label='y=3')
plt.plot(kappa11,delta_b_star_4[4,:],'g',label='y=2')
plt.plot(kappa11,delta_b_star_4[3,:],'r',label='y=1')
plt.plot(kappa11,delta_b_star_4[2,:],'c',label='y=-1')
plt.plot(kappa11,delta_b_star_4[1,:],'violet',label='y=-2')
plt.plot(kappa11,delta_b_star_4[0,:],'y',label='y=-3')
plt.legend(loc='center right')
plt.xlabel('$\\kappa^1$')
#plt.ylim(-1, 1)  # Set the limits of y-axis
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
#plt.yticks([0,0.005, 0.01, 0.015,0.02]);  # Set specific y-axis ticks

# If you need to set ticks in Times New Roman, ensure this by setting tick labels explicitly if needed
# Example to set x-ticks (similarly for y-ticks):
plt.xticks(fontname='Times New Roman')
plt.yticks(fontname='Times New Roman')
plt.savefig('General analysis Model 1.b.png', format='png', dpi=300, bbox_inches='tight')